In [91]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time, lxml, requests, re
import pandas as pd
from gensim.summarization import summarize
from googletrans import Translator

# 팝업 전담 일진 : 거지같은 팝업 제거 성공!!!!!! 안녕 지겨웠고 다신 보지 말자!
def kill_iframe(driver):
    iframe = driver.find_element_by_class_name('truste_popframe')
    # 이게 키포인트! iframe에 포커스하면 iframe 안의 엘리먼트를 찾을 수 있습니다!
    driver.switch_to.frame(iframe)
    target = driver.find_element_by_link_text('모두 거절')
    target.click()
    time.sleep(5)
    target = driver.find_element_by_link_text('닫기')
    target.click()

def main_process(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    #print(soup.prettify())

    #타이틀
    title = soup.find('h1', class_='title').get_text()
    title = re.sub('[\s]+', ' ', title)
    print(title.split(' '))
    category = title.split(' ')[0]
    count = 0
    select_lists = []
    print(category)
    swit = -1 # 대분류를 나타내는 변수

    # 현재 페이지가 클래스나 인터페이스인 경우(첫번째 대분류)
    if category == 'Class' or category == 'Interface':
        swit = 1 # 대분류 첫번째 표식
        titles = soup.findAll('div', class_='sub-title')

        # 모듈
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 패키지
        package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, package])
        
        # 본문
        content = soup.find('div', class_='block').get_text()

        # 상세 : 구현 인터페이스, 구현 클래스 등......
        # 구현한 인터페이스
        section = soup.find('section', class_='description')
        imple_inter_raw = section.find('dl', class_='notes')
        imple_inter_title = imple_inter_raw.find('dt').get_text()
        imple_inters = imple_inter_raw.select('dd > code > a')
        imple_inters = [a.get_text() for a in imple_inters]
        imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[0].find_elements_by_tag_name('a')
        if category == 'Class': imple_inter = ['구현한 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
        else: imple_inter = ['구현 클래스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
        count += 1
        select_lists.append([count, imple_inter])

        # 서브 클래스
        if len(section.findAll('dl', class_='notes')) > 1:
            known_sub_raw = section.findAll('dl', class_='notes')[1]
            known_sub_title = known_sub_raw.find('dt').get_text()
            known_subs = known_sub_raw.select('dd > code > a')
            known_subs = [a.get_text() for a in known_subs]
            known_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[1].find_elements_by_tag_name('a')
            known_sub = ['서브 클래스', [[name, target] for name, target in zip(known_subs, known_targets)]]

            count += 1
            select_lists.append([count, known_sub])

        sig_raw = soup.find('div', class_='type-signature').get_text()
        sig = re.sub('\n', ' ', sig_raw)

        # 중첩 클래스
        nested_raw = soup.find('section', id='nested.class.summary')
        if nested_raw != None:
            nest_title = nested_raw.find('h2').get_text()
            nest_table = nested_raw.find('div', class_='summary-table three-column-summary')
            if nest_table != None:
                nest_col1 = nest_table.findAll('div', class_='col-first')
                nest_col2 = nest_table.findAll('div', class_='col-second')
                nest_col3 = nest_table.findAll('div', class_='col-last')

                nest_col1 = [target.get_text() for target in nest_col1]
                nest_col2 = [target.get_text() for target in nest_col2]
                nest_col3 = [re.sub('\n', ' ', target.get_text()) for target in nest_col3]

                nest_target = driver.find_element_by_id('nested.class.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                nest_target = [scol.find_elements_by_tag_name('a') for scol in nest_target] #
                
                nest_df = pd.DataFrame([nest_col1[1:], nest_col2[1:], nest_col3[1:], nest_target[1:]]).T
                nest = ['중첩 클래스', nest_df] #######

                count += 1
                select_lists.append([count, nest])

        # 필드
        field_raw = soup.find('section', id='field.summary')
        if field_raw != None:
            field_title = field_raw.find('h2').get_text()
            field_table = field_raw.find('div', class_='summary-table three-column-summary')
            
            if field_table != None:
                field_col1 = field_table.findAll('div', class_='col-first')
                field_col2 = field_table.findAll('div', class_='col-second')
                field_col3 = field_table.findAll('div', class_='col-last')

                field_col1 = [target.get_text() for target in field_col1]
                field_col2 = [target.get_text() for target in field_col2]
                field_col3 = [re.sub('\n', ' ', target.get_text()) for target in field_col3]

                field_target = driver.find_element_by_id('field.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                field_target = [scol.find_elements_by_tag_name('a') for scol in field_target] #
                
                field_df = pd.DataFrame([field_col1[1:], field_col2[1:], field_col3[1:], field_target[1:]]).T
                field = ['필드', field_df] #######

                count += 1
                select_lists.append([count, field])

        # 생성자
        const_cols = 0
        const_sel = -1 # 별거 없음. 자기 자신의 번호를 여기다 저장. 나중에 선택할 때 생성자를 선택했다는 식별용 -1은 생성자 없다는 뜻
        const_raw = soup.find('section', id='constructor.summary')
        if const_raw != None:
            const_title = const_raw.find('h2').get_text()
            
            const_table = const_raw.find('div', class_='summary-table two-column-summary')
            if const_table == None: # 컬럼이 3개인 생성자 목록일 때
                const_table = const_raw.find('div', class_='summary-table three-column-summary')
                if const_table != None: 
                    const_cols = 3
                    const_first = const_table.findAll('div', class_='col-first')[1:]
                    const_name = const_table.findAll('div', class_='col-constructor-name')
                    const_last = const_table.findAll('div', class_='col-last')[1:]
                    
                    const_first = [target.get_text() for target in const_first]
                    const_name = [target.get_text() for target in const_name]
                    const_last = [re.sub('\n', ' ', target.get_text()) for target in const_last]
                    
                    const_target = driver.find_element_by_id('constructor.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-constructor-name')
                    const_target = [scol.find_elements_by_tag_name('a') for scol in const_target] #
                    
                    const_df = pd.DataFrame([const_first, const_name, const_last, const_target]).T
                    const = ['생성자', const_df] #######
                    
                    count += 1
                    const_sel = count
                    select_lists.append([count, const])  
            else: # 컬럼이 2개인 생성자 목록일 때
                const_cols = 2
                const_name = const_table.findAll('div', class_='col-constructor-name') # 이름 : 바로 생성자들부터 시작
                const_last = const_table.findAll('div', class_='col-last')[1:] # 설명 : 컬럼 이름부터 시작 : 그래서 1부터 슬라이싱

                const_name = [target.get_text() for target in const_name]
                const_last = [re.sub('\n', ' ', target.get_text()) for target in const_last]

                const_target = driver.find_element_by_id('constructor.summary').find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-constructor-name')
                const_target = [scol.find_elements_by_tag_name('a') for scol in const_target] #
                
                const_df = pd.DataFrame([const_name, const_last, const_target]).T
                const = ['생성자', const_df] #######

                count += 1
                const_sel = count
                select_lists.append([count, const])
                #print(const)

        # 메서드
        method_raw = soup.find('section', id='method.summary')
        if method_raw != None:
            method_title = method_raw.find('h2').get_text()
            method_table = method_raw.find('div', class_='summary-table three-column-summary')
            
            if method_table != None:
                method_col1 = method_table.findAll('div', class_='col-first')
                method_col2 = method_table.findAll('div', class_='col-second')
                method_col3 = method_table.findAll('div', class_='col-last')

                method_col1 = [target.get_text() for target in method_col1]
                method_col2 = [re.sub('\n', ' ', target.get_text()) for target in method_col2]
                method_col3 = [re.sub('\n', ' ', target.get_text()) for target in method_col3]

                method_target = driver.find_element_by_id('method.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                method_target = [scol.find_elements_by_tag_name('a') for scol in method_target]
                
                method_df = pd.DataFrame([method_col1[1:], method_col2[1:], method_col3[1:], method_target[1:]]).T
                method = ['메서드', method_df] #######

                #밑의 방법으로 들어갈 수 있음.
                #method_target[1][0].click()
                count += 1
                select_lists.append([count, method])

        
    
    # 현재 페이지가 패키지인 경우
    elif category == 'Package': 
        swit = 2 # 대분류 두번째 표식
        titles = soup.findAll('div', class_='sub-title')
        
        # 모듈 : 패키지는 무조건 모듈밖에 없음!
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # 인터페이스, 클래스, 열거형, 에러, 예외, 에너테이션 인터페이스를 모아놓은 summary-list
        summary_list = soup.find('ul', class_='summary-list')
        summary_lis = summary_list.findAll('li')
        midx = -1
        for li in summary_lis:
            midx += 1
            # 분기할 기준. 제목을 가지고 종류를 분별.
            caption = li.find('div', class_='caption').get_text()
            # 각각 종류별 목록들이 있는 테이블.
            table = li.find('div', class_='summary-table two-column-summary')
            
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            
            # 인터페이스 목록
            if caption == 'Interface Summary':
                #########################################################
                # 셀레니움으로 첫번째 열까지 찾는다. 여기에 링크가 있다. 마찬가지로, 첫번째는 그냥 칼럼 이름이기 때문에 슬라이싱 해준다.
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                # targets[원하는 항목 인덱스][0]에 click()나 send_keys(Keys.ENTER)을 쓰면 된다.
                ########################################################
                inter_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                inter = ['인터페이스', inter_df]
                count += 1
                select_lists.append([count, inter])
            
            # 클래스 목록
            elif caption == 'Class Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                classs_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                classs = ['클래스', classs_df]
                count += 1
                select_lists.append([count, classs])
            
            # 열거형 목록
            elif caption == 'Enum Class Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                enumc_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                enumc = ['열거형', enumc_df]
                count += 1
                select_lists.append([count, enumc])
            
            # 예외 목록
            elif caption == 'Exception Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                exc_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                exc = ['예외', exc_df]
                count += 1
                select_lists.append([count, exc])
                
            # 에러 목록
            elif caption == 'Error Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                errors_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                errors = ['에러', errors_df]
                count += 1
                select_lists.append([count, errors])
                
            # 애너테이션 인터페이스 목록
            elif caption == 'Annotation Interfaces Summary':
                targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_elements_by_class_name('col-first')[1:]
                col_targets = [target.find_elements_by_tag_name('a') for target in targets]
                ais_df = pd.DataFrame([col_first, col_last, col_targets]).T    
                ais = ['애너테이션 인터페이스', ais_df]
                count += 1
                select_lists.append([count, ais])
    
    # 현재 페이지가 모듈인 경우
    elif category == 'Module': 
        swit = 3 # 대분류 세번째 표식
        mod_tab = -1
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # 패키지 모음 : inderect Exports는 하위에 있응 모듈에서 들어가도 나오므로 Exports만 크롤링
        package_raw = soup.find('section', id='packages.summary')
        if package_raw != None:
            table = package_raw.find('div', class_='summary-table two-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('packages.summary').find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            package_df = pd.DataFrame([col_first, col_last, col_targets]).T
            packages = ['패키지', package_df]
            count += 1
            select_lists.append([count, packages])
            
        # 모듈 모음 : 모듈은 컬럼이 3개이다.
        module_raw = soup.find('section', id='modules.summary')
        if module_raw != None:
            table = module_raw.find('div', class_='details-table three-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. modifier : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_second = [col.get_text() for col in col_second]
            col_last = table.findAll('div', class_='col-last')[1:] # 마지막 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('modules.summary').find_elements_by_class_name('details-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            module_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
            modules = ['모듈', module_df]
            count += 1
            mod_tab = count
            select_lists.append([count, modules])
            
        # 서비스 모음
        service_raw = soup.find('section', id='services.summary')
        if service_raw != None:
            table = service_raw.find('div', class_='details-table two-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('services.summary').find_elements_by_class_name('details-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            service_df = pd.DataFrame([col_first, col_last, col_targets]).T
            services = ['서비스', service_df]
            count += 1
            select_lists.append([count, services])
    
    # 현재 페이지가 애너테이션 인터페이스인 경우 : 선택적 요소와 필수 요소로 나누인다.
    elif category == 'Annotation': 
        swit = 4 # 대분류 네번째 표식
        titles = soup.findAll('div', class_='sub-title')

        # 모듈
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 패키지
        package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, package])
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # optional element
        optional_element_raw = soup.find('section', id='annotation.interface.optional.element.summary')
        if optional_element_raw != None:
            print('optional')
            table = optional_element_raw.find('div', class_='summary-table three-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. modifier, 타입 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_second = [col.get_text() for col in col_second]
            col_last = table.findAll('div', class_='col-last')[1:] # 마지막 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('annotation.interface.optional.element.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            oe_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
            oes = ['선택적 요소', oe_df]
            count += 1
            select_lists.append([count, oes])
            
        # required element
        required_element_raw = soup.find('section', id='annotation.interface.required.element.summary')
        if required_element_raw != None:
            print('required')
            table = required_element_raw.find('div', class_='summary-table three-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. modifier, 타입 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_second = table.findAll('div', class_='col-second')[1:] # 두번째 열. 이름 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_second = [col.get_text() for col in col_second]
            col_last = table.findAll('div', class_='col-last')[1:] # 마지막 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            targets = driver.find_element_by_id('annotation.interface.required.element.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            
            req_df = pd.DataFrame([col_first, col_second, col_last, col_targets]).T
            req = ['필수 요소', req_df]
            count += 1
            select_lists.append([count, req])
    
    # 현재 페이지가 열거형 클래스인 경우
    elif category == 'Enum': 
        swit = 5 # 대분류 다섯번째 표식
        titles = soup.findAll('div', class_='sub-title')

        # 모듈
        module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, module])
        
        # 패키지
        package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
        count += 1
        select_lists.append([count, package])
        
        # 구현한 인터페이스
        section = soup.find('section', class_='description')
        imple_inter_raw = section.find('dl', class_='notes')
        imple_inter_title = imple_inter_raw.find('dt').get_text()
        imple_inters = imple_inter_raw.select('dd > code > a')
        imple_inters = [a.get_text() for a in imple_inters]
        imple_targets = driver.find_elements_by_class_name('description')[0].find_elements_by_class_name('notes')[0].find_elements_by_tag_name('a')
        if category == 'Class': imple_inter = ['구현한 인터페이스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
        else: imple_inter = ['구현 클래스', [[name, target] for name, target in zip(imple_inters, imple_targets)]]
        count += 1
        select_lists.append([count, imple_inter])
        
        # 본문
        content = soup.find('div', class_='block').get_text()
        
        # 중첩 클래스
        nested_raw = soup.find('section', id='nested.class.summary')
        if nested_raw != None:
            wrap = nested_raw.find('code')
            wrap = wrap.get_text() if wrap != None else '없음'
            target = driver.find_element_by_id('nested.class.summary').find_elements_by_tag_name('a')[0]
            nested = ['중첩 클래스', wrap, target]
            count += 1
            select_lists.append([count, nested])
            
        # 열거 상수
        two_row = -1
        constant_raw = soup.find('section', id='enum.constant.summary')
        if constant_raw != None:
            table = constant_raw.find('div', class_='summary-table two-column-summary')
            col_first = table.findAll('div', class_='col-first')[1:] # 첫번째 열. 이름 : 첫번째 행은 그냥 칼럼 이름이다.
            col_first = [col.get_text() for col in col_first]
            col_last = table.findAll('div', class_='col-last')[1:] # 두번째 열. 설명 : 역시 첫번째 행은 그냥 칼럼 이름이다.
            col_last = [col.get_text() for col in col_last]
            
            # 셀레니움으로 첫번째 열까지 찾는다. 여기에 링크가 있다. 마찬가지로, 첫번째는 그냥 칼럼 이름이기 때문에 슬라이싱 해준다.
            targets = driver.find_elements_by_id('enum.constant.summary')[0].find_elements_by_class_name('summary-table.two-column-summary')[0].find_elements_by_class_name('col-first')[1:]
            col_targets = [target.find_elements_by_tag_name('a') for target in targets]
            # targets[원하는 항목 인덱스][0]에 click()나 send_keys(Keys.ENTER)을 쓰면 된다.
            constant_df = pd.DataFrame([col_first, col_last, col_targets]).T
            constants = ['열거 상수', constant_df] #######
            count += 1
            two_row = count
            select_lists.append([count, constants])
            
        # 메서드
        method_raw = soup.find('section', id='method.summary')
        if method_raw != None:
            method_table = method_raw.find('div', class_='summary-table three-column-summary')
            
            if method_table != None:
                method_col1 = method_table.findAll('div', class_='col-first')
                method_col2 = method_table.findAll('div', class_='col-second')
                method_col3 = method_table.findAll('div', class_='col-last')

                method_col1 = [target.get_text() for target in method_col1]
                method_col2 = [re.sub('\n', ' ', target.get_text()) for target in method_col2]
                method_col3 = [re.sub('\n', ' ', target.get_text()) for target in method_col3]

                method_target = driver.find_element_by_id('method.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                method_target = [scol.find_elements_by_tag_name('a') for scol in method_target]
                
                method_df = pd.DataFrame([method_col1[1:], method_col2[1:], method_col3[1:], method_target[1:]]).T
                method = ['메서드', method_df] #######

                #밑의 방법으로 들어갈 수 있음.
                #method_target[1][0].click()
                count += 1
                select_lists.append([count, method])
                
    
    # 카카오톡으로 전송
    count += 1
    select_lists.append([count, ['카카오톡으로 전송']])
    
    # 본문 번역기 돌리기
    trans = Translator()
    at = trans.translate(content, src='en', dest='ko')
    if 100 < len(at.text) <= 300:
        summ = summarize(at.text, ratio=0.8)
    elif 300 < len(at.text) <= 700:
        summ = summarize(at.text, ratio=0.5)
    elif 700 < len(at.text):
        summ = summarize(at.text, ratio=0.2)
    else: summ = at.text
        
    # 사랑의 작대기
    while True:
        print()
        print(title)
        if swit in [1, 4, 5]: 
            sub_info = '({} : {} / {} : {})'.format(module[0], module[1], package[0], package[1])
            print(sub_info)
        elif swit == 2: 
            sub_info = '({} : {})'.format(module[0], module[1])
            print(sub_info)
        print()
        print(summ)
        print()
        print('메뉴')
        for count, sel in select_lists:
            print(count, '. ', sel[0], sep='', end='\t')
        sel = int(input('원하시는 메뉴를 선택해주세요. ====> '))
            
        if swit == 1: # 대분류 첫번째이면 여기로
            # 무조건 method = ['메서드', method_df]가 통째로 들어간다. 필드, 메서드도 마찬가지!
            if const_sel != -1 and sel == const_sel: # 생성자가 있고, 생성자를 선택했다면
                if const_cols == 2: # 근데, 생성자 목록 칼럼 수가 2개라면
                    res = select_cat(select_lists[sel-1][1], cols=2)
            elif select_lists[sel-1][1][0] in ['모듈', '패키지'] :
                return [0, select_lists[sel-1][1]]
            else: # 어쨌건 생성자를 선택 안했다면
                res = select_cat(select_lists[sel-1][1])
        elif swit == 2: # 대분류 두번째이면 여기로
            res = select_cat(select_lists[sel-1][1], cols=2, swit=2)
        elif swit == 3: # 대분류 세번째이면 여기로
            if sel == mod_tab: res = select_cat(select_lists[sel-1][1], swit=3)
            else: res = select_cat(select_lists[sel-1][1], cols=2, swit=2)
        elif swit == 4: # 대분류 네번째이면 여기로!
            res = select_cat(select_lists[sel-1][1])
        elif swit == 5:
            print('선택한 항목 : ', select_lists[sel-1][0])
            if select_lists[sel-1][1][0] in ['모듈', '패키지', '중첩 클래스']: return [0, select_lists[sel-1][1]]
            elif sel == two_row : res = select_cat(select_lists[sel-1][1], cols=2)
            else: res = select_cat(select_lists[sel-1][1])
            
        # res = [결과 코드, 결과 데이터(행)] 
        # 1. 구현 클래스, 구현한 인터페이스, 서브클래스
        # 2. 필드, 메서드, 생성자
        # 3. 중첩 클래스, 모듈 페이지 일부
        # 4. 패키지 페이지
        # 7. 카카오톡으로 전송(전송 후 종료)
        if sel == count: return [7, title, sub_info, summ]
        elif res[0] == 0: continue
        elif res[0] == 1: return res
        elif res[0] == 2: return res
        elif res[0] == 3: return res
        elif res[0] == 4: return res
    
def select_cat(select_list, cols=-1, swit=-1):
    print(select_list)
    count = 0
    if select_list[0] in ['구현 클래스', '구현한 인터페이스', '서브 클래스']:
        print('vun 1')
        print('\n', select_list[0])
        for idx, li in enumerate(select_list[1]):
            print((idx+1), ' : ', li[0])
            count = idx+1
        print((count + 1), ' : ', '돌아가기')
        sel = int(input('2-0. 원하시는 메뉴를 선택해주세요. ====> '))
        if sel == count + 1: return [0, 0]
        else: return [1, select_list[1][sel-1]] # 구현 클래스, 구현한 인터페이스, 서브클래스이면 결과 코드를 1로
    # 필드, 생성자, 메서드
    elif cols == 2: # 칼럼이 2개일 때 (이건 거의 생성자 특수용. 필드나 메서드는 대부분 칼럼이 3개이다.)
        print('col 2')
        print('\n', select_list[0])
        for i in range(len(select_list[1])):
            count += 1
            row = select_list[1].iloc[i]
            print((count), ' : ', row[0], row[1], '\n')
        print((count + 1), ' : ', '돌아가기')
        sel = int(input('2. 원하시는 메뉴를 선택해주세요. ====> ')) 
        if sel == count + 1: return [0, 0]
        else:
            if swit == 2: return [4, select_list[1].iloc[sel-1]] # 패키지 페이지이면 결과 코드를 4로
            else: return [2, select_list[1].iloc[sel-1]]
    else: # 칼럼이 3개!
        print('col 3')
        print('\n', select_list[0])
        for i in range(len(select_list[1])):
            count += 1
            
            row = select_list[1].iloc[i]
            print((count), ' : ', row[0], row[1], '\n', row[2], '\n')
        print((count + 1), ' : ', '돌아가기')
        sel = int(input('2. 원하시는 메뉴를 선택해주세요. ====> ')) 
        if sel == count + 1: return [0, 0]
        else: 
            if select_list[0] == '중첩 클래스' or swit == 3: return [3, select_list[1].iloc[sel-1]] # 중첩 클래스이거나 모듈 페이지이면 결과 코드를 3으로
            else: return [2, select_list[1].iloc[sel-1]] # 필드, 메서드, 생성자이면 결과 코드를 2로
    
    
# 메인 로직
kakaoTalk = False
driver_path = './resource/WebDriver/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(driver_path)
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/Frame.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/image/ImageObserver.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/javax/swing/JApplet.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/javax/swing/package-summary.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/module-summary.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/beans/ConstructorProperties.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/beans/BeanProperty.html'
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/GraphicsDevice.WindowTranslucency.html'
test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/package-summary.html'
driver.get(test_url)
time.sleep(4)

#kill_iframe(driver)
result = ''
while True:
    result = main_process(driver)
    if result[0] == 0: result[1][2].send_keys(Keys.ENTER)
    elif result[0] == 1: result[1][1].send_keys(Keys.ENTER)
    elif result[0] == 2: # 필드, 메서드, 생성자, 
        if len(result[1]) == 4: result[1][3][0].click()
        else: result[1][2][0].click()
        break
    elif result[0] == 3: # 중첩 클래스일 때
        result[1][3][0].click()
    elif result[0] == 4: # 패키지 페이지에서 선택했을 때, 즉, swit == 2일 때
        result[1][2][0].click()
  #  elif result[0] == 5:
   #     result[1][3][0].click()
    elif result[0] == 7:
        kakaoTalk = True
        break
        
if kakaoTalk == True:
    print()
    print(result[1])
    print()
    print(result[2])
    print()
    print(result[3])
    print()
    print('='*30, 'The End', '='*30)

C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:606: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


['Package', 'java.awt']
Package


C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:210: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:210: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:237: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_ele


Package java.awt
(모듈 : java.desktop)

사용자 인터페이스 생성 및 페인팅을위한 모든 클래스가 포함되어 있습니다.
모든 AWT 구성 요소가 공유합니다.
AWT 이벤트 모델에 대한 설명은 Awtevent를 참조하십시오.
AWT 패키지에는 여러 레이아웃이 포함되어 있습니다
이러한 최대 값을 변경할 방법이 없습니다.
플랫폼 제한, 컨테이너 내에 올바르게 배열 할 수있는 방법이 없습니다.
Java AWT Native Interface (Jawt)

메뉴
1. 모듈	2. 인터페이스	3. 클래스	4. 열거형	5. 예외	6. 에러	7. 카카오톡으로 전송	원하시는 메뉴를 선택해주세요. ====> 4
['열거형',                                        0  \
0       Component.BaselineResizeBehavior   
1                         Desktop.Action   
2              Dialog.ModalExclusionType   
3                    Dialog.ModalityType   
4      GraphicsDevice.WindowTranslucency   
5   MultipleGradientPaint.ColorSpaceType   
6      MultipleGradientPaint.CycleMethod   
7                        Taskbar.Feature   
8                          Taskbar.State   
9                   TrayIcon.MessageType   
10                           Window.Type   

                                                    1  \
0   \nEnumeration of the common ways the baseline ...   
1 

C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:412: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:412: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:417: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
C:\Use

...


Class IllegalComponentStateException
(모듈 : java.desktop / 패키지 : java.awt)

AWT 구성 요소가 적절한 상태에 있지 않다는 신호
 요청 된 작업.

메뉴
1. 모듈	2. 패키지	3. 구현한 인터페이스	4. 서브 클래스	5. 생성자	6. 카카오톡으로 전송	원하시는 메뉴를 선택해주세요. ====> 2
['Package', 'java.awt']
Package


C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:210: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:210: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:237: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  targets = driver.find_elements_by_class_name('summary-list')[0].find_elements_by_class_name('summary-table.two-column-summary')[midx].find_ele


Package java.awt
(모듈 : java.desktop)

사용자 인터페이스 생성 및 페인팅을위한 모든 클래스가 포함되어 있습니다.
모든 AWT 구성 요소가 공유합니다.
AWT 이벤트 모델에 대한 설명은 Awtevent를 참조하십시오.
AWT 패키지에는 여러 레이아웃이 포함되어 있습니다
이러한 최대 값을 변경할 방법이 없습니다.
플랫폼 제한, 컨테이너 내에 올바르게 배열 할 수있는 방법이 없습니다.
Java AWT Native Interface (Jawt)

메뉴
1. 모듈	2. 인터페이스	3. 클래스	4. 열거형	5. 예외	6. 에러	7. 카카오톡으로 전송	원하시는 메뉴를 선택해주세요. ====> 6
['에러',           0                                                  1  \
0  AWTError  \nThrown when a serious Abstract Window Toolki...   

                                                   2  
0  [<selenium.webdriver.remote.webelement.WebElem...  ]
col 2

 에러
1  :  AWTError 
Thrown when a serious Abstract Window Toolkit error has occurred.
 

2  :  돌아가기
2. 원하시는 메뉴를 선택해주세요. ====> 1
['Class', 'AWTError']
Class


C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:41: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:41: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  module = ['모듈', titles[0].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[0].find_elements_by_tag_name('a')[0]]
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\54646647.py:46: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  package = ['패키지', titles[1].find('a').get_text(), driver.find_elements_by_class_name('sub-title')[1].find_elements_by_tag_name('a')[0]]
C:\Users\


Class AWTError
(모듈 : java.desktop / 패키지 : java.awt)

심각한 추상 창 툴킷 오류가 발생했을 때 발생했습니다.

메뉴
1. 모듈	2. 패키지	3. 구현한 인터페이스	4. 서브 클래스	5. 생성자	6. 카카오톡으로 전송	원하시는 메뉴를 선택해주세요. ====> 5
['생성자',                        0                                                  1  \
0  AWTError​(String msg)   Constructs an instance of AWTError with the s...   

                                                   2  
0  [<selenium.webdriver.remote.webelement.WebElem...  ]
col 2

 생성자
1  :  AWTError​(String msg)  Constructs an instance of AWTError with the specified  detail message.  

2  :  돌아가기
2. 원하시는 메뉴를 선택해주세요. ====> 1


In [9]:
# 실험용 로직. 메인 로직이 아닙니다!

def main_process(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    #print(soup.prettify())

    #타이틀
    title = soup.find('h1', class_='title').get_text()
    category = title.split(' ')[0]
    count = 0
    select_lists = []
    
    if category == 'Class' or category == 'Interface':
        titles = soup.findAll('div', class_='sub-title')
        
        
        method_raw = soup.find('section', id='method.summary')
        if method_raw != None:
            method_title = method_raw.find('h2').get_text()
            method_table = method_raw.find('div', class_='summary-table three-column-summary')
            
            if method_table != None:
                method_col1 = method_table.findAll('div', class_='col-first')
                method_col2 = method_table.findAll('div', class_='col-second')
                method_col3 = method_table.findAll('div', class_='col-last')

                method_col1 = [target.get_text() for target in method_col1]
                method_col2 = [re.sub('\n', ' ', target.get_text()) for target in method_col2]
                method_col3 = [re.sub('\n', ' ', target.get_text()) for target in method_col3]
                print(method_col1, end='\n\n')
                print(method_col2, end='\n\n')
                print(method_col3, end='\n\n')
                
                print('method_target')
                method_target = driver.find_element_by_id('method.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
                method_target = [scol.find_elements_by_tag_name('a') for scol in method_target]
                print(method_target, end='\n\n')
                method_target[1][0].click()
                
                method_df = pd.DataFrame([method_col1[1:], method_col2[1:], method_col3[1:], method_target[1:]]).T
                method = ['메서드', method_df]

                #print(method)
                
                
                count += 1
                select_lists.append([count, method])
                #print(method)
    

test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/image/ImageObserver.html'
    
kakaoTalk = False
driver_path = './resource/WebDriver/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(driver_path)
#test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/Frame.html'
test_url = 'https://docs.oracle.com/en/java/javase/16/docs/api/java.desktop/java/awt/image/ImageObserver.html'
driver.get(test_url)
time.sleep(4)

kill_iframe(driver)
result = ''
result = main_process(driver)

C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\3398465844.py:54: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\2329648415.py:11: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  iframe = driver.find_element_by_class_name('truste_popframe')
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\2329648415.py:14: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  target = driver.find_element_by_link_text('모두 거절')


['Modifier and Type', 'boolean']

['Method', 'imageUpdate\u200b(Image\xa0img,  int\xa0infoflags,  int\xa0x,  int\xa0y,  int\xa0width,  int\xa0height)']

['Description', ' This method is called when information about an image which was  previously requested using an asynchronous interface becomes  available. ']

method_target
[[], [<selenium.webdriver.remote.webelement.WebElement (session="dc27415f7d175865ad7be59c99b348d7", element="7cbbd1d6-c498-4560-9803-e360797853a0")>, <selenium.webdriver.remote.webelement.WebElement (session="dc27415f7d175865ad7be59c99b348d7", element="2393f668-8df5-4a3f-aac0-cae77d9c4058")>]]



C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\2329648415.py:17: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  target = driver.find_element_by_link_text('닫기')
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\3398465844.py:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  method_target = driver.find_element_by_id('method.summary').find_elements_by_class_name('summary-table.three-column-summary')[0].find_elements_by_class_name('col-second')
C:\Users\Mercedes\AppData\Local\Temp\ipykernel_11096\3398465844.py:35: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  method_target = [scol.find_elements_by_tag_name('a') for scol in method_target]
